In [1]:
import torch
import numpy as np
from copy import copy, deepcopy
from tqdm import tqdm

## Tug of War 2 lanes No FIFO

In [3]:
maker_cost = {
    'Marine T' : 50,
    'Baneling T' : 75,
    'Immortal T' : 200,
    'Marine B' : 50,
    'Baneling B' : 75,
    'Immortal B' : 200,
}
def get_big_A_spend_all(miner, 
              all_A_vectors = None, vector = None, move = 0):
    if all_A_vectors is None:
        all_A_vectors = list()
    if vector is None:
        vector = (0,0,0,0,0,0)
    if miner == 0:
        all_A_vectors.append(vector)
        return list(all_A_vectors)
    
    next_vector = copy(vector)
#     if miner < 50:
#         all_A_vectors.append(vector)
#         return list(all_A_vectors)
        
#     next_vector = copy(vector)
#     get_big_A_spend_all(miner - miner, all_A_vectors, next_vector)
    if miner >= maker_cost['Marine T']:
        if move <= 0:
            next_vector = (vector[0] + 1, vector[1],vector[2],
                            vector[3], vector[4], vector[5])
            get_big_A_spend_all(miner - maker_cost['Marine T'], all_A_vectors, next_vector, 0)
        if move <= 1:
            next_vector = (vector[0], vector[1],vector[2],
                            vector[3] + 1, vector[4], vector[5])
            get_big_A_spend_all(miner - maker_cost['Marine B'], all_A_vectors, next_vector, 1)
            
        if miner >= maker_cost['Baneling T']:
            if move <= 2:
                next_vector = (vector[0], vector[1] + 1,vector[2],
                                vector[3], vector[4], vector[5])
                get_big_A_spend_all(miner - maker_cost['Baneling T'], all_A_vectors, next_vector, 2)
            if move <= 3:
                next_vector = (vector[0], vector[1],vector[2],
                                vector[3], vector[4] + 1, vector[5])
                get_big_A_spend_all(miner - maker_cost['Baneling B'], all_A_vectors, next_vector, 3)
                
            if miner >= maker_cost['Immortal T']:
                if move <= 4:
                    next_vector = (vector[0], vector[1],vector[2] + 1,
                                    vector[3], vector[4], vector[5])
                    get_big_A_spend_all(miner - maker_cost['Immortal T'], all_A_vectors, next_vector, 4)
                if move <= 5:
                    next_vector = (vector[0], vector[1],vector[2],
                                    vector[3], vector[4], vector[5] + 1)
                    get_big_A_spend_all(miner - maker_cost['Immortal B'], all_A_vectors, next_vector, 5)

    return list(all_A_vectors)

In [4]:
# action_space = get_big_A_spend_all(225)
# print(len(action_space))
# for action in action_space:
#     print(action)

all_actions = []
mineral = 0
maker_cost_np = np.zeros(len(maker_cost))
for i, mc in enumerate(maker_cost.values()):
    maker_cost_np[i] = mc
for i in tqdm(range(1500 // 25 + 1)):
    all_actions.extend(get_big_A_spend_all(mineral))
    mineral += 25
    if mineral > 1500:
        break


 95%|█████████▌| 58/61 [00:03<00:00,  8.03it/s]


In [18]:
all_actions = np.array(all_actions)
pylon_action = np.zeros((all_actions.shape[0], 1))
print(all_actions.shape)
print(pylon_action.shape)
all_actions = np.hstack((all_actions, pylon_action))
print(all_actions.shape)
print(all_actions)
# print(all_actions[all_actions[:,5] == 7])

(91692, 6)
(91692, 1)
(91692, 7)
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 2. ... 4. 4. 0.]
 [0. 0. 1. ... 4. 5. 0.]
 [0. 0. 0. ... 4. 6. 0.]]


In [21]:
key_mineral = 50
action_dict = {0: 1, 25: 1}
for i, a in tqdm(enumerate(np.array(all_actions[1:]))):
#     if key_mineral < 500:
#         print(np.sum(maker_cost_np * a))
    if np.sum(maker_cost_np * a[:-1]) != key_mineral:
        action_dict[key_mineral] = i + 1
        key_mineral += 25
action_dict[key_mineral] = i + 2
print(action_dict)
action_1500_dict = {}
all_actions_torch = torch.Tensor(all_actions)
action_1500_dict['actions'] = all_actions_torch
action_1500_dict['mineral'] = action_dict

torch.save(action_1500_dict, 'action_1500_dict_2L.pt')

91691it [00:00, 291313.34it/s]

{0: 1, 25: 1, 50: 3, 75: 5, 100: 8, 125: 12, 150: 19, 175: 25, 200: 38, 225: 50, 250: 69, 275: 91, 300: 121, 325: 153, 350: 200, 375: 248, 400: 315, 425: 387, 450: 481, 475: 583, 500: 714, 525: 854, 550: 1031, 575: 1221, 600: 1457, 625: 1709, 650: 2018, 675: 2350, 700: 2747, 725: 3175, 750: 3683, 775: 4225, 800: 4867, 825: 5551, 850: 6348, 875: 7204, 900: 8189, 925: 9241, 950: 10451, 975: 11737, 1000: 13205, 1025: 14769, 1050: 16538, 1075: 18422, 1100: 20543, 1125: 22795, 1150: 25319, 1175: 27997, 1200: 30984, 1225: 34148, 1250: 37663, 1275: 41385, 1300: 45497, 1325: 49849, 1350: 54642, 1375: 59702, 1400: 65263, 1425: 71127, 1450: 77543, 1475: 84313, 1500: 91692}


## Tug of War 2 lanes No FIFO (One lane one time)

In [3]:
maker_cost = {
    'Marine' : 50,
    'Baneling' : 75,
    'Immortal' : 200,
#     'Plyon' : 300
}
def get_big_A_spend_all_olot(miner, 
              all_A_vectors = None, vector = None, move = 0, pylons = 0):
    if all_A_vectors is None:
        all_A_vectors = list()
    if vector is None:
        vector = (0,0,0,0)
    if miner == 0:
        all_A_vectors.append(vector) 
        return list(all_A_vectors)
    pylon_cost = 300 + pylons * 100
    next_vector = copy(vector)
#     if miner < 50:
#         all_A_vectors.append(vector)
#         return list(all_A_vectors)
        
#     next_vector = copy(vector)
#     get_big_A_spend_all_olot(miner - miner, all_A_vectors, next_vector)
    if miner >= maker_cost['Marine']:
        if move <= 0:
            next_vector = (vector[0] + 1, vector[1],vector[2], vector[3])
            get_big_A_spend_all_olot(miner - maker_cost['Marine'], all_A_vectors, next_vector, 0,
                                     pylons = pylons)
            
        if miner >= maker_cost['Baneling']:
            if move <= 1:
                next_vector = (vector[0], vector[1] + 1,vector[2], vector[3])
                get_big_A_spend_all_olot(miner - maker_cost['Baneling'], all_A_vectors, next_vector, 1,
                                         pylons = pylons)
                
            if miner >= maker_cost['Immortal']:
                if move <= 2:
                    next_vector = (vector[0], vector[1],vector[2] + 1, vector[3])
                    get_big_A_spend_all_olot(miner - maker_cost['Immortal'], all_A_vectors, next_vector, 2,
                                             pylons = pylons)
                if miner >= pylon_cost and pylons < 3:
                    if move <= 3:
                        next_vector = (vector[0], vector[1], vector[2], vector[3] + 1)
                        get_big_A_spend_all_olot(miner - pylon_cost, all_A_vectors, next_vector, 3, 
                                                 pylons = pylons + 1)

    return list(all_A_vectors)

In [4]:
# action_space = get_big_A_spend_all_olot(1500)
# print(len(action_space))
# for action in action_space:
#     print(action)
maker_cost_np = np.zeros(len(maker_cost))
for i, mc in enumerate(maker_cost.values()):
    maker_cost_np[i] = mc
    
all_actions = {}
for i in range(4):
    actions = []
    mineral = 0
    for _ in tqdm(range(1500 // 25 + 1)):
        actions.extend(get_big_A_spend_all_olot(mineral, pylons = i))
        mineral += 25
        if mineral > 1500:
            break
    
    print(len(actions))
    all_actions[i] = np.array(actions)

  0%|          | 0/61 [00:00<?, ?it/s]

1843
1601
1399
1041


In [5]:
for i in range(4):
    print(i)
    print(all_actions[i][-20:])

0
[[ 2  8  4  0]
 [ 2  4  4  1]
 [ 2  4  2  2]
 [ 2  0  7  0]
 [ 2  0  1  3]
 [ 1 14  2  0]
 [ 1 10  2  1]
 [ 1 10  0  2]
 [ 1  6  5  0]
 [ 1  2  5  1]
 [ 1  2  3  2]
 [ 0 20  0  0]
 [ 0 16  0  1]
 [ 0 12  3  0]
 [ 0  8  3  1]
 [ 0  8  1  2]
 [ 0  4  6  0]
 [ 0  4  0  3]
 [ 0  0  6  1]
 [ 0  0  4  2]]
1
[[ 3  2  6  0]
 [ 3  2  4  1]
 [ 2 16  1  0]
 [ 2  8  4  0]
 [ 2  8  2  1]
 [ 2  4  1  2]
 [ 2  0  7  0]
 [ 2  0  5  1]
 [ 1 14  2  0]
 [ 1 14  0  1]
 [ 1  6  5  0]
 [ 1  6  3  1]
 [ 1  2  2  2]
 [ 0 20  0  0]
 [ 0 12  3  0]
 [ 0 12  1  1]
 [ 0  8  0  2]
 [ 0  4  6  0]
 [ 0  4  4  1]
 [ 0  0  3  2]]
2
[[ 4  4  5  0]
 [ 4  0  4  1]
 [ 3 18  0  0]
 [ 3 10  3  0]
 [ 3  6  2  1]
 [ 3  2  6  0]
 [ 2 16  1  0]
 [ 2 12  0  1]
 [ 2  8  4  0]
 [ 2  4  3  1]
 [ 2  0  7  0]
 [ 1 14  2  0]
 [ 1 10  1  1]
 [ 1  6  5  0]
 [ 1  2  4  1]
 [ 0 20  0  0]
 [ 0 12  3  0]
 [ 0  8  2  1]
 [ 0  4  6  0]
 [ 0  0  5  1]]
3
[[ 7 10  2  0]
 [ 7  2  5  0]
 [ 6 16  0  0]
 [ 6  8  3  0]
 [ 6  0  6  0]
 [ 5 14  1  0]

In [6]:
pylon_cost = {
    0:{
        0: 0,
        1: 300,
        2: 700,
        3: 1200
    },
    1:{
        0: 0,
        1: 400,
        2: 900
    },
    2:{
        0: 0,
        1: 500
    },
    3: {
        0: 0,
    }
}
all_action_dict = {}
for j in range(4):
    action_pylon = all_actions[j]
    all_action_dict[j] = {0: 1, 25: 1}
    key_mineral = 50
    for i, a in tqdm(enumerate(np.array(action_pylon[1:]))):
        pylon_c = pylon_cost[j][a[-1]]
#         print(pylon_c)
#         print(a)
#         if key_mineral < 500:
#         print(np.sum(maker_cost_np * a[:-1]) + pylon_c)
        if np.sum(maker_cost_np * a[:-1]) + pylon_c != key_mineral:
            all_action_dict[j][key_mineral] = i + 1
            key_mineral += 25
            
    all_action_dict[j][key_mineral] = i + 2
    print(all_action_dict)
    
action_1500_dict = {}
action_1500_dict['actions'] = all_actions
action_1500_dict['mineral'] = all_action_dict

torch.save(action_1500_dict, 'action_1500_dict_2L.pt')

1842it [00:00, 151830.75it/s]
1600it [00:00, 160309.74it/s]
1398it [00:00, 156538.98it/s]
1040it [00:00, 152594.84it/s]

{0: {0: 1, 25: 1, 50: 2, 75: 3, 100: 4, 125: 5, 150: 7, 175: 8, 200: 11, 225: 13, 250: 16, 275: 19, 300: 24, 325: 27, 350: 33, 375: 38, 400: 45, 425: 51, 450: 60, 475: 67, 500: 78, 525: 87, 550: 99, 575: 110, 600: 125, 625: 137, 650: 154, 675: 169, 700: 189, 725: 206, 750: 229, 775: 249, 800: 275, 825: 298, 850: 327, 875: 353, 900: 387, 925: 416, 950: 453, 975: 487, 1000: 528, 1025: 565, 1050: 611, 1075: 652, 1100: 703, 1125: 749, 1150: 804, 1175: 855, 1200: 917, 1225: 972, 1250: 1039, 1275: 1101, 1300: 1174, 1325: 1241, 1350: 1321, 1375: 1394, 1400: 1481, 1425: 1561, 1450: 1654, 1475: 1741, 1500: 1843}}
{0: {0: 1, 25: 1, 50: 2, 75: 3, 100: 4, 125: 5, 150: 7, 175: 8, 200: 11, 225: 13, 250: 16, 275: 19, 300: 24, 325: 27, 350: 33, 375: 38, 400: 45, 425: 51, 450: 60, 475: 67, 500: 78, 525: 87, 550: 99, 575: 110, 600: 125, 625: 137, 650: 154, 675: 169, 700: 189, 725: 206, 750: 229, 775: 249, 800: 275, 825: 298, 850: 327, 875: 353, 900: 387, 925: 416, 950: 453, 975: 487, 1000: 528, 1025: 56

In [7]:
print(action_1500_dict)

{'actions': {0: array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 4, 0, 3],
       [0, 0, 6, 1],
       [0, 0, 4, 2]]), 1: array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 4, 6, 0],
       [0, 4, 4, 1],
       [0, 0, 3, 2]]), 2: array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 8, 2, 1],
       [0, 4, 6, 0],
       [0, 0, 5, 1]]), 3: array([[ 0,  0,  0,  0],
       [ 1,  0,  0,  0],
       [ 0,  1,  0,  0],
       ...,
       [ 0, 20,  0,  0],
       [ 0, 12,  3,  0],
       [ 0,  4,  6,  0]])}, 'mineral': {0: {0: 1, 25: 1, 50: 2, 75: 3, 100: 4, 125: 5, 150: 7, 175: 8, 200: 11, 225: 13, 250: 16, 275: 19, 300: 24, 325: 27, 350: 33, 375: 38, 400: 45, 425: 51, 450: 60, 475: 67, 500: 78, 525: 87, 550: 99, 575: 110, 600: 125, 625: 137, 650: 154, 675: 169, 700: 189, 725: 206, 750: 229, 775: 249, 800: 275, 825: 298, 850: 327, 875: 353, 900: 387, 925: 416, 950: 453, 975: 487, 1000: 528,

In [8]:
a = np.array([1,2,3])


## Generate Marked experience.

In [2]:
unmark_exp = torch.load("test_random_vs_random_2l.pt")
def player_1_win_condition(state_1_T_hp, state_1_B_hp, state_2_T_hp, state_2_B_hp):
    if min(state_1_T_hp, state_1_B_hp) == min(state_2_T_hp, state_2_B_hp):
        if state_1_T_hp + state_1_B_hp == state_2_T_hp + state_2_B_hp:
            return 0
        elif state_1_T_hp + state_1_B_hp > state_2_T_hp + state_2_B_hp:
            return 1
        else:
            return -1
    else:
        if min(state_1_T_hp, state_1_B_hp) > min(state_2_T_hp, state_2_B_hp):
            return 1
        else:
            return -1
mark_exp = deepcopy(unmark_exp)

In [3]:
insert_num = 0
for i, exp in enumerate(unmark_exp):
#     print(exp)
    if i != 0 and exp[0][-1] == 2:
        s1_hp_t, s1_hp_b, s2_hp_t, s2_hp_b = unmark_exp[i - 1][1][27:31]
        
        win_lose = player_1_win_condition(s1_hp_t, s1_hp_b, s2_hp_t, s2_hp_b)
        mark_exp.insert(i + insert_num, win_lose)
#         skip = True
        insert_num += 1
#         print(s1_hp_t, s1_hp_b, s2_hp_t, s2_hp_b)
#         print(win_lose)


In [5]:
for i, exp in enumerate(mark_exp):
    print(exp)
    if type(exp) == type(int(1)):
        print(list(mark_exp[i - 1][1][27:31]))

[array([2.5e+02, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 1.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 2.0e+03,
       2.0e+03, 2.0e+03, 2.0e+03, 2.0e+00]), array([2.5e+02, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 1.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 1.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 2.0e+03,
       2.0e+03, 2.0e+03, 2.0e+03, 2.0e+00, 0.0e+00])]
[array([1.5e+02, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 1.0e+00,
       0.0e+00, 0.0e+00, 1.0e+00, 0.0e+00, 2.0e+00, 1.0e+00, 0.0e+00,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       0.0e+00, 1.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 2.0e+03,
       2.0e+03, 2.0e+03, 2.0e+03, 3.0e+00]), array([ 1.50e+

In [6]:
torch.save(mark_exp, "test_random_vs_random_2l_mark.pt")